In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

%matplotlib inline
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

In [66]:
noah_training = pd.read_csv('train_election_demographic.csv')
noah_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Columns: 165 entries, State to early_voting
dtypes: float64(55), int64(101), object(9)
memory usage: 4.0+ MB


In [67]:
noah_training.head()

,State,ST,Fips_x,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Republicans 2012,Republicans 2008,Democrats 2012,Democrats 2008,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,...,"Poverty Percent, Age 0-17","Poverty Estimate, Age 5-17 in Families","Poverty Percent, Age 5-17 in Families",Median Household Income,NIPR,NUI,NIC,PCTUI,PCTIC,PCTELIG,PCTLIIC,--,Administrative,Agriculture,Construction,Educational Services,Finance,Food services,Health care,Information,Management,Manufacturing,Mining,Other,Professional Services,Real Estate,Recreation,Retail,Transportation,Utitilities,Wholesale,FIPStxt_y,Fips,County.1,2012 Population,2016 Population,state population 2012,state population 2016,% of state pop 2012,% of state pop 2016,# of state advanced ballots cast 2012,# of state advanced ballots cast 2016,calculated approx. county adv. Ballots 2012,calculated approx. county adv. Ballots 2016,# votes 2012,# votes 2016,id_laws,reg_laws,felony_laws,early_voting
0,Georgia,GA,13089,"DeKalb County, Georgia",192.0,295871.0,254594.0,238224.0,65581.0,64392.0,16.064771,80.822723,NaN,3.112505,21.024929,20.347560,77.783619,78.991883,50,13,89,Georgia,DeKalb County,9,0,746802,353157,393645,131573,135656,187973,223367,2088,1591,23936,24325,287,322,7300,8384,137315,142129,192299,228617,4568,4582,26410,27094,642,713,...,25.8,29054,24.4,56053,643155,98243,544912,15.3,84.7,15.3,84.7,16822,4640.0,35.0,3920.0,1750.0,4325.0,7485.0,10010.0,1910.0,695.0,2010.0,30.0,7325.0,14950.0,5090.0,1460.0,11225.0,2295.0,145.0,4640.0,13089,13089,"DeKalb County, Georgia",710557,746802,9901430,10301890,0.071763,0.072492,1918931,2381782,137708.376900,172659.537300,306265.0,314757.0,3.0,3.0,1.0,0.0
1,Texas,TX,48487,"Wilbarger County, Texas",7.0,4101.0,1196.0,971.0,3283.0,2956.0,77.200683,19.678127,0.316996,2.804194,74.271357,72.809936,24.396985,26.524728,50,48,487,Texas,Wilbarger County,9,0,12880,6393,6487,5407,5556,534,515,99,103,203,179,5,5,145,129,5537,5674,602,584,159,152,229,200,11,10,...,24.7,486,23.6,40955,10423,2012,8411,19.3,80.7,19.3,80.7,294,30.0,10.0,100.0,5.0,150.0,165.0,145.0,45.0,NaN,55.0,35.0,180.0,90.0,60.0,20.0,225.0,55.0,40.0,55.0,48487,48487,"Wilbarger County, Texas",13289,12880,26084481,27914410,0.000509,0.000461,5182127,4497431,2640.086483,2075.161584,3980.0,4105.0,3.0,2.0,0.0,0.0
2,Virginia,VA,51111,"Lunenburg County, Virginia",12.0,5577.0,2703.0,2684.0,2900.0,2969.0,57.486104,39.913932,0.448270,1.577909,51.778863,51.327434,46.808511,47.840708,50,51,111,Virginia,Lunenburg County,9,0,12319,6523,5796,4033,3760,2316,1854,29,29,15,22,12,9,118,122,4139,3875,2389,1925,62,76,40,37,17,14,...,26.1,414,25.5,38313,8651,1234,7417,14.3,85.7,14.3,85.7,173,40.0,25.0,135.0,NaN,45.0,45.0,75.0,10.0,5.0,35.0,NaN,125.0,40.0,10.0,10.0,190.0,40.0,NaN,35.0,51111,51111,"Lunenburg County, Virginia",12638,12319,8185080,8410106,0.001544,0.001465,447907,541543,691.581349,793.244249,5734.0,5586.0,2.0,2.0,2.0,1.0
3,Georgia,GA,13297,"Walton County, Georgia",21.0,40425.0,8469.0,8148.0,27253.0,29036.0,76.915275,20.479901,NaN,2.604824,77.198766,75.648143,21.663299,23.508022,50,13,297,Georgia,Walton County,9,0,89786,43724,46062,34879,36013,7273,8409,170,184,591,703,52,42,759,711,35560,36648,7680,8809,406,387,773,869,87,88,...,18.9,2994,17.7,55267,76272,10825,65447,14.2,85.8,14.2,85.8,1602,590.0,30.0,1415.0,105.0,445.0,555.0,745.0,95.0,25.0,250.0,5.0,825.0,720.0,405.0,80.0,980.0,290.0,30.0,405.0,13297,13297,"Walton County, Georgia",84975,89786,9901430,10301890,0.008582,0.008715,1918931,2381782,16468.445640,20758.392750,37612.0,40698.0,3.0,3.0,1.0,0.0
4,North Carolina,NC,37011,"Avery County, North Carolina",19.0,8060.0,2178.0,1882.0,56

In [68]:
saul_training = pd.read_csv('merged_covid_financial_2016.csv')
saul_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3139 entries, 0 to 3138
Columns: 159 entries, State to senate_dem_cash_on_hand
dtypes: float64(64), int64(87), object(8)
memory usage: 3.8+ MB


In [69]:
intersect_pred = list(noah_training.columns.intersection(saul_training.columns))

In [70]:
saul_pred = saul_training.columns.difference(noah_training.columns)

In [71]:
noah_pred = list(noah_training.columns.difference(saul_training.columns))
noah_pred.append('Fips_x')
noah_pred

['# of state advanced ballots cast 2012',
 '# of state advanced ballots cast 2016',
 '# votes 2012',
 '# votes 2016',
 '% of state pop 2012',
 '% of state pop 2016',
 '2012 Population',
 '2016 Population',
 'County.1',
 'Fips',
 'calculated approx. county adv. Ballots 2012',
 'calculated approx. county adv. Ballots 2016',
 'early_voting',
 'felony_laws',
 'id_laws',
 'reg_laws',
 'state population 2012',
 'state population 2016',
 'Fips_x']

In [72]:
training_df = pd.merge(saul_training, noah_training[noah_pred], how='left', on=['Fips_x'])

In [73]:
list(training_df.columns)

['State',
 'ST',
 'Fips_x',
 'County',
 'Precincts',
 'Votes',
 'Democrats 08 (Votes)',
 'Democrats 12 (Votes)',
 'Republicans 08 (Votes)',
 'Republicans 12 (Votes)',
 'Republicans 2016',
 'Democrats 2016',
 'Green 2016',
 'Libertarians 2016',
 'Republicans 2012',
 'Republicans 2008',
 'Democrats 2012',
 'Democrats 2008',
 'SUMLEV',
 'STATE',
 'COUNTY',
 'STNAME',
 'CTYNAME',
 'YEAR',
 'AGEGRP',
 'TOT_POP',
 'TOT_MALE',
 'TOT_FEMALE',
 'WA_MALE',
 'WA_FEMALE',
 'BA_MALE',
 'BA_FEMALE',
 'IA_MALE',
 'IA_FEMALE',
 'AA_MALE',
 'AA_FEMALE',
 'NA_MALE',
 'NA_FEMALE',
 'TOM_MALE',
 'TOM_FEMALE',
 'WAC_MALE',
 'WAC_FEMALE',
 'BAC_MALE',
 'BAC_FEMALE',
 'IAC_MALE',
 'IAC_FEMALE',
 'AAC_MALE',
 'AAC_FEMALE',
 'NAC_MALE',
 'NAC_FEMALE',
 'NH_MALE',
 'NH_FEMALE',
 'NHWA_MALE',
 'NHWA_FEMALE',
 'NHBA_MALE',
 'NHBA_FEMALE',
 'NHIA_MALE',
 'NHIA_FEMALE',
 'NHAA_MALE',
 'NHAA_FEMALE',
 'NHNA_MALE',
 'NHNA_FEMALE',
 'NHTOM_MALE',
 'NHTOM_FEMALE',
 'NHWAC_MALE',
 'NHWAC_FEMALE',
 'NHBAC_MALE',
 'NHBAC_

In [74]:
training_df['County.1']

0        Barbour County, Alabama
1         Morgan County, Alabama
2        Houston County, Alabama
3          Macon County, Alabama
4        Bullock County, Alabama
                  ...           
3134      Goshen County, Wyoming
3135    Big Horn County, Wyoming
3136       Uinta County, Wyoming
3137     Lincoln County, Wyoming
3138    Sublette County, Wyoming
Name: County.1, Length: 3139, dtype: object

In [75]:
# 
training_df.rename(columns={'Fips_x':'FIPS'}, inplace=True)
training_df.County = training_df.CTYNAME
drop_pred = ['STATE', 'COUNTY', 'STNAME', 'CTYNAME','FIPS', 'Fips_y','FIPStxt_x',\
                                'Stabr', 'area_name', 'FIPStxt_y', 'County.1', 'Fips']

training_df = training_df.drop(drop_pred, axis = 1)
training_df.head()

,State,ST,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Republicans 2012,Republicans 2008,Democrats 2012,Democrats 2008,SUMLEV,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,...,PCTELIG,PCTLIIC,--,Administrative,Agriculture,Construction,Educational Services,Finance,Food services,Health care,Information,Management,Manufacturing,Mining,Other,Professional Services,Real Estate,Recreation,Retail,Transportation,Utitilities,Wholesale,pres_rep_receipts,pres_rep_disburse,pres_rep_cash_on_hand,pres_dem_receipts,pres_dem_disburse,pres_dem_cash_on_hand,senate_rep_receipts,senate_rep_disburse,senate_rep_cash_on_hand,senate_dem_receipts,senate_dem_disburse,senate_dem_cash_on_hand,# of state advanced ballots cast 2012,# of state advanced ballots cast 2016,# votes 2012,# votes 2016,% of state pop 2012,% of state pop 2016,2012 Population,2016 Population,calculated approx. county adv. Ballots 2012,calculated approx. county adv. Ballots 2016,early_voting,felony_laws,id_laws,reg_laws,state population 2012,state population 2016
0,Alabama,AL,Barbour County,18.0,10390.0,5697.0,5912.0,5866.0,5550.0,52.271415,46.660250,0.173244,0.895091,48.189633,50.438521,51.332812,48.985383,50,9,0,25806,13626,12180,6779,6071,6515,5858,91,60,54,58,28,13,159,120,6897,6176,6622,5929,168,107,69,78,33,18,13004,11689,6272,5665,6473,...,12.5,87.5,452.0,30.0,65.0,115.0,25.0,210.0,255.0,215.0,25.0,NaN,135.0,20.0,180.0,160.0,100.0,25.0,440.0,145.0,40.0,75.0,205.0,5271.0,0.0,0.0,0.0,0.0,5914160.0,13077935.0,9992806.0,33268.0,32369.0,442.0,0,2399,11509.0,10469.0,0.005642,0.005306,27169,25806,0.0,12.729161,2.0,1.0,3.0,2.0,4815588,4863525
1,Alabama,AL,Morgan County,42.0,49991.0,13895.0,13439.0,36014.0,35391.0,74.797464,22.436038,0.436078,2.330419,71.556239,71.255589,27.172001,27.491987,50,9,0,119006,58586,60420,48940,50152,7353,7871,669,609,354,461,93,73,1177,1254,50052,51320,7890,8446,1245,1217,509,628,118,116,53272,55978,44185,46234,7124,...,11.2,88.8,2573.0,580.0,25.0,1020.0,80.0,860.0,1030.0,1690.0,145.0,70.0,790.0,20.0,1375.0,940.0,500.0,130.0,2355.0,405.0,20.0,810.0,205.0,5271.0,0.0,0.0,0.0,0.0,5914160.0,13077935.0,9992806.0,33268.0,32369.0,442.0,0,2399,49323.0,50644.0,0.024940,0.024469,120102,119006,0.0,58.701332,2.0,1.0,3.0,2.0,4815588,4863525
2,Alabama,AL,Houston County,29.0,42030.0,12225.0,12367.0,29254.0,29270.0,72.726624,25.093980,0.342612,1.836783,69.717035,70.094645,29.456460,29.291961,50,9,0,104218,50012,54206,35320,37258,12975,15037,251,279,411,595,35,44,1020,993,36237,38129,13523,15566,627,654,605,782,90,112,48208,52631,33865,36022,12851,...,11.1,88.9,2704.0,605.0,25.0,975.0,95.0,965.0,1260.0,1585.0,220.0,115.0,515.0,NaN,1385.0,1055.0,505.0,140.0,2710.0,425.0,30.0,880.0,205.0,5271.0,0.0,0.0,0.0,0.0,5914160.0,13077935.0,9992806.0,33268.0,32369.0,442.0,0,2399,41918.0,42639.0,0.021466,0.021428,103371,104218,0.0,51.406949,2.0,1.0,3.0,2.0,4815588,4863525
3,Alabama,AL,Macon County,16.0,8748.0,9450.0,9045.0,1396.0,1331.0,15.935071,82.727481,0.548697,0.788752,12.803001,12.834421,87.004617,86.880574,50,9,0,19060,8710,10350,1559,1552,6935,8579,21,29,75,28,2,1,118,161,1647,1675,7006,8666,71,90,99,72,10,17,8535,10197,1471,1492,6878,...,10.4,89.6,200.0,55.0,5.0,50.0,15.0,45.0,105.0,120.0,15.0,NaN,30.0,10.0,160.0,55.0,25.0,10.0,230.0,40.0,NaN,30.0,205.0,5271.0,0.0,0.0,0.0,0.0,5914160.0,13077935.0,9992806.0,33268.0,32369.0,442.0,0,2399,10394.0,9140.0,0.004279,0.003919,20608,19060,0.0,9.401605,2.0,1.0,3.0,2.0,4815588,4863525
4,Alabama,AL,Bullock County,17.0,4701.0,4011.0,4061.0,1391.0,1251.0,24.228887,75.090406,0.212721,0.467986,23.506201,25.687904,76.305900,74.072022,50,9,0,10389,5653,4736,1671,1074,3818,3

In [76]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3139 entries, 0 to 3138
Columns: 164 entries, State to state population 2016
dtypes: float64(74), int64(86), object(4)
memory usage: 4.0+ MB


In [77]:
training_df.to_csv('~/Desktop/training_set.csv')